# Tool Windows: A Declarative Approach

Tool windows are child windows of the IDE that allow to display additional information or provide always-visible user interface elements. As seen in the [programmatic Tool Window example](tool_window.ipynb), the tool window elements can be configured in the Kotlin or Java code.

Alternatively, we can use _a declarative_ approach by using plugin descriptor and an extension point.

Let's mirror the previous example and create a simple tool window with the current time.

![Tool Window with current time](tool_window/tool_window_right.png)

## Enabling IntelliJ Platform Integration

In [2]:
%use intellij-platform

IntelliJ Platform integration is loaded

### Tool Window Content Class

 `CurrentTimeToolWindowContent` contains an actual user interface elements.

In [3]:
import java.awt.BorderLayout
import java.time.ZonedDateTime
import java.time.format.DateTimeFormatter
import javax.swing.BorderFactory
import javax.swing.JLabel
import javax.swing.JPanel
import javax.swing.SwingConstants

class CurrentTimeToolWindowContent {
    val contentPanel = JPanel()

    private val currentTime = JLabel().apply {
        horizontalAlignment = SwingConstants.CENTER
    }

    init {
        with(contentPanel) {
            layout = BorderLayout(0, 20)
            border = BorderFactory.createEmptyBorder(40, 0, 0, 0)
            add(currentTime, BorderLayout.CENTER)
        }
        updateCurrentTime()
    }

    fun updateCurrentTime() {
        currentTime.text = ZonedDateTime.now().format(TIME_FORMATTER)
    }

    companion object {
        private val TIME_FORMATTER = DateTimeFormatter.ofPattern("HH:mm")
    }
}

## Tool Window Factory

A tool window factory creates a user interface elements and registers it into an instance of `ToolWindow` provided by IntelliJ Platform API.

In [4]:
import com.intellij.openapi.project.DumbAware
import com.intellij.openapi.project.Project
import com.intellij.openapi.wm.ToolWindow
import com.intellij.openapi.wm.ToolWindowFactory
import com.intellij.ui.content.ContentFactory

class CurrentTimeToolWindowFactory : ToolWindowFactory, DumbAware {
    override fun createToolWindowContent(project: Project, toolWindow: ToolWindow) {
        val toolWindowContent = CurrentTimeToolWindowContent()
        val content = ContentFactory.getInstance().createContent(
            toolWindowContent.contentPanel, "", false
        )
        toolWindow.contentManager.addContent(content)
    }
}

## Registering the Tool Window Factory



In a regular plugin, registering a tool window factory requires declaring an extension in the plugin descriptor.

```xml
<extensions defaultExtensionNs="com.intellij">
    <toolWindow id="Current Time"
                factoryClass="somePackage.CurrentTimeToolWindowFactory"
                anchor="right"
                icon="/general/information.svg" />
</extensions>
```

This XML declaration can be mirrored in Kotlin code. We will create an instance of the corresponding extension point.
Then, we will register this instance using `registerExtension` function.

In [5]:
import com.intellij.openapi.extensions.ExtensionPointName
import com.intellij.openapi.wm.ToolWindowAnchor
import com.intellij.openapi.wm.ToolWindowEP

runInEdt {
    val toolWindowEP = ToolWindowEP().apply {
        id = "Current Time"
        factoryClass = CurrentTimeToolWindowFactory::class.java.name
        anchor = ToolWindowAnchor.RIGHT.toString()
        icon = "AllIcons.General.Information"
        pluginDescriptor = notebookPluginDescriptor
    }
    val toolWindowEpName = ExtensionPointName.create<ToolWindowEP>("com.intellij.toolWindow")

    registerExtension(toolWindowEpName, toolWindowEP)
}


There are few important aspects of this configuration.

### Referring to a Tool Window Factory

The `ToolWindowEP` instance of an extension point refers to a _factory class_. This is not an instance of the tool window factory class, but a fully qualified class name represented as a string.

Since the instance is declared in this Kotlin Notebook, we will refer it by `class.java.name` member. This will provide a correct implicit package name and necessary technical class name prefixes provided by Kotlin Notebook integration.

### Plugin Descriptor

The extension point instance requires a reference to the plugin descriptor. While not necessary in XML, this is required in programmatic declaration. An instance of type-safe plugin descriptor is provided by the `notebookPluginDescriptor` variable. Additionally, this instance contains properly initialized classloaders that will load the tool window factory class.

### Anchors and Icons

Since we mirror the XML declarations, both anchor and icons are provided as strings. The `AllIcons.General.Information` refers to a constant provided by the Platform SDK.

## Registering an extension

The built-in function `registerExtension` will register an extension into the platform. Furthermore, it will ensure a proper lifecycle and disposing of tool window factories and tool windows when the Kernel is restarted.

## Summary

- Refer to the [Tool Window](tool_window.ipynb) example for basic concepts.
- Use `::class.java.name` to refer to a fully qualified class name that is declared in the notebook.
- Use `notebookPluginDescriptor` to refer to the plugin descriptor instance.
- Use `registerExtension` to register an extension into the Platform with correct disposal management.